In [18]:
import requests
import pandas as pd
import sqlalchemy
from bs4 import BeautifulSoup
import re

In [78]:
url = 'https://www.otodom.pl/sprzedaz/mieszkanie/warszawa/'
params = {'search[filter_float_price:to]':'400000',
          'search[filter_float_m:from]': '25', 
          'search[description]': '1', 
          'search[order]': 'created_at_first:desc',
          'search[dist]': '0',
          'search[subregion_id]': '197', 
          'search[city_id]': '26'}

In [85]:
r = requests.get(url, params=params)
r_html = r.text
soup = BeautifulSoup(r_html, 'html.parser')

In [80]:
offer_items = soup.find_all('div','offer-item-details')

In [81]:
re_district = re.compile(r'(Mieszkanie na sprzedaż: Warszawa,) (.*)')
re_digits = re.compile(r'\s*(\d*\s*\d*) (.*)')
re_digits.search('6 228 zł/m²').group(1)

'6 228'

In [82]:
offers = dict()
for num, offer in enumerate(offer_items, start=1):
    title = offer.find('span', 'offer-item-title').text.strip()
    district = re_district.search(offer.find('p', 'text-nowrap hidden-xs').text).group(2).strip()
    room = re_digits.search(offer.find('li', 'offer-item-rooms hidden-xs').text).group(1).strip()
    price = re.sub(' ', '', re_digits.search(offer.find('li', 'offer-item-price').text).group(1)).strip()
    area = re.sub(' ', '', re_digits.search(offer.find('li', 'hidden-xs offer-item-area').text).group(1)).strip()
    price_per_m = re.sub(' ', '', re_digits.search(offer.find('li', 'hidden-xs offer-item-price-per-m').text).group(1)).strip()
    offers[num] = {'title': title, 'district': district, 'rooms': room, 'price': price, 'area': area,
                  'price_per_m': price_per_m}

{1: {'title': 'Mieszkanie 2 pokoje, Targówek, 100 m Metro Trocka',
  'district': 'Targówek Mieszkaniowy',
  'rooms': '2',
  'price': '350000',
  'area': '30',
  'price_per_m': '6958'},
 2: {'title': 'Mieszkanie 47 m2 Wola',
  'district': 'Wola',
  'rooms': '2',
  'price': '399000',
  'area': '47',
  'price_per_m': '8489'},
 3: {'title': 'Okazja ! Pilne ! Metro !!!',
  'district': 'Targówek',
  'rooms': '2',
  'price': '399800',
  'area': '56',
  'price_per_m': '7139'},
 4: {'title': 'METRO CITY LINK Mieszkanie przy stacji metra! Wola',
  'district': 'Śródmieście',
  'rooms': '1',
  'price': '376066',
  'area': '87',
  'price_per_m': '11800'},
 5: {'title': '2 pokoje z 70 m2 tarasem, 5min do Metra',
  'district': 'Nowa Praga',
  'rooms': '2',
  'price': '400000',
  'area': '77',
  'price_per_m': '11845'},
 6: {'title': 'Mieszkanie z ogródkiem na Woli, gotowa do odbioru!',
  'district': 'Odolany',
  'rooms': '1',
  'price': '349000',
  'area': '31',
  'price_per_m': '11258'},
 7: {'title